In [1]:
#!/usr/bin/env python
# coding: utf-8
from sqlalchemy import create_engine
import sys
import platform
import logging
import trino

sys.path.append('..')
from common import db_operations

import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from common.db_operations import connect_to_trino, fetch_data_for_day, write_df_to_iceberg

from math import radians, sin, cos, sqrt, atan2

In [2]:
# Create SQLAlchemy engine for Trino
# engine = create_engine('trino://admin@trino:8080/adhoc/default')

In [3]:
# Configure basic logging for the business logic file
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

# Print the Python version being used
print(f"Using Python version: {platform.python_version()}")

Using Python version: 3.11.13


In [4]:
conn = trino.dbapi.connect(
    host="trino",
    port=8080,
    user="admin",
    catalog="adhoc",
    schema="default"
)

# Create SQLAlchemy engine for Trino
engine = create_engine('trino://admin@trino:8080/adhoc/default')

In [5]:
import trino
import pandas as pd
from datetime import datetime, timedelta

# Connect to Trino
conn = trino.dbapi.connect(
    host="trino",
    port=8080,
    user="admin",
    catalog="adhoc",
    schema="default"
)

# Fix the SQL query - change 'minutes' to 'minute'
query = """
with cpo100 as
(
  SELECT 
    id, timestamp,dt, 
    date_trunc('minute', timestamp) as ts_mins,  -- Truncate to minutes
    batterycoolingstate,batterycoolanttemperature,ignitionstatus,
    temperaturedifferencealarm,chargingcurrentalarm,dischargecurrentalarm,
    vehiclereadycondition,gun_connection_status,ignitionstatus,
    pack1_cellmax_temperature,pack1_maxtemperature_cell_number,
    pack1_cell_min_temperature,pack1_celltemperature_cellnumber
  from 
    facts_prod.can_parsed_output_100
  where 
    -- timestamp::date between date('2025-10-01') and date('2025-10-02')
    timestamp >= now() - interval '30' minute  -- Fixed: 'minutes' to 'minute'
    and id in ('3','16','18')
),
cpo100_ranked as
(
  select 
    *,row_number()over(partition by id,ts_mins order by timestamp) as cpo_rn
  from 
    cpo100
),
can_ac as
(
  select 
    id, timestamp,date,
    date_trunc('minute', timestamp) as ts_mins_cac,    
    b2t_tms_control_cmd,b2t_set_water_out_temp,b2t_battery_min_temp,b2t_battery_max_temp,
    tms_working_mode,tms_fault_code,coolant_out_temp,coolant_in_temp,
    (coolant_out_temp - coolant_in_temp) as coolant_temp_diff,
    comp_target_hz as comp_target_freq,
    comp_running_frequency as comp_running_freq,
    comp_status,v2t_vehicle_coolant_low,comp_current
  from 
    facts_prod.can_output_ac
  where
    -- timestamp::date between date('2025-10-01') and date('2025-10-02')
    timestamp >= now() - interval '30' minute  -- Fixed: 'minutes' to 'minute'
    -- timestamp::date between date('2025-10-01') and date('2025-10-02') and
    -- timestamp >= now() - interval '1' hour  -- Filter for last 1 hour and
    -- timestamp >= CAST('2025-10-07' AS TIMESTAMP) AT TIME ZONE 'Asia/Kolkata' - INTERVAL '5' HOUR - INTERVAL '30' MINUTE and
    -- and timestamp < CAST('2025-10-09' AS TIMESTAMP) AT TIME ZONE 'Asia/Kolkata' - INTERVAL '5' HOUR - INTERVAL '30' MINUTE and   
    and id in ('3','16','18')
    and b2t_battery_min_temp != -40
    and coolant_in_temp != 0
),
can_ac_ranked as
(
  select 
    *, row_number() over(partition by id,ts_mins_cac order by timestamp) as cac_rn
  from can_ac
)
select *
from 
  cpo100_ranked as cp
  right join
  can_ac_ranked as cn
  on (cp.id = cn.id and cp.ts_mins = cn.ts_mins_cac and cp.cpo_rn = cn.cac_rn)
"""

,id,timestamp,dt,ts_mins,batterycoolingstate,batterycoolanttemperature,ignitionstatus,temperaturedifferencealarm,chargingcurrentalarm,dischargecurrentalarm,...,tms_fault_code,coolant_out_temp,coolant_in_temp,coolant_temp_diff,comp_target_freq,comp_running_freq,comp_status,v2t_vehicle_coolant_low,comp_current,cac_rn
0,16,2025-10-10 11:53:01.810,2025-10-10,2025-10-10 11:53:00,None,None,1.0,0.0,0.0,0.0,...,No Fault,24,24,0,46.0,46.0,On,Normal,NaN,1
1,16,2025-10-10 11:53:02.770,2025-10-10,2025-10-10 11:53:00,None,None,1.0,0.0,0.0,0.0,...,No Fault,24,24,0,NaN,NaN,None,Normal,NaN,2
2,16,2025-10-10 11:53:03.810,2025-10-10,2025-10-10 11:53:00,None,None,NaN,0.0,0.0,0.0,...,No Fault,24,24,0,NaN,NaN,None,Normal,NaN,3
3,16,2025-10-10 11:53:04.750,2025-10-10,2025-10-10 11:53:00,None,None,1.0,0.0,0.0,0.0,...,No Fault,23,24,-1,NaN,NaN,None,Normal,NaN,4
4,16,2025-10-10 11:53:06.375,2025-10-10,2025-10-10 11:53:00,None,None,1.0,0.0,0.0,0.0,...,No Fault,23,24,-1,NaN,NaN,None,Normal,NaN,5


Index(['id', 'timestamp', 'dt', 'ts_mins', 'batterycoolingstate',
       'batterycoolanttemperature', 'ignitionstatus',
       'temperaturedifferencealarm', 'chargingcurrentalarm',
       'dischargecurrentalarm', 'vehiclereadycondition',
       'gun_connection_status', 'ignitionstatus', 'pack1_cellmax_temperature',
       'pack1_maxtemperature_cell_number', 'pack1_cell_min_temperature',
       'pack1_celltemperature_cellnumber', 'cpo_rn', 'id', 'timestamp', 'date',
       'ts_mins_cac', 'b2t_tms_control_cmd', 'b2t_set_water_out_temp',
       'b2t_battery_min_temp', 'b2t_battery_max_temp', 'tms_working_mode',
       'tms_fault_code', 'coolant_out_temp', 'coolant_in_temp',
       'coolant_temp_diff', 'comp_target_freq', 'comp_running_freq',
       'comp_status', 'v2t_vehicle_coolant_low', 'comp_current', 'cac_rn'],
      dtype='object')

232


In [6]:
# Use Trino cursor directly instead of pandas.read_sql
cur = conn.cursor()
cur.execute(query)

# Get column names
columns = [desc[0] for desc in cur.description]

# Fetch all rows
rows = cur.fetchall()

# Create DataFrame manually
df = pd.DataFrame(rows, columns=columns)

# Display results
display(df.head())
display(df.columns)
print(len(df))

# Close the cursor and connection
cur.close()
conn.close()

,id,timestamp,dt,ts_mins,batterycoolingstate,batterycoolanttemperature,ignitionstatus,temperaturedifferencealarm,chargingcurrentalarm,dischargecurrentalarm,...,tms_fault_code,coolant_out_temp,coolant_in_temp,coolant_temp_diff,comp_target_freq,comp_running_freq,comp_status,v2t_vehicle_coolant_low,comp_current,cac_rn
0,16,2025-10-10 12:08:00.210,2025-10-10,2025-10-10 12:08:00,None,None,NaN,0.0,0.0,0.0,...,No Fault,23,24,-1,NaN,NaN,None,Normal,NaN,1
1,16,2025-10-10 12:08:01.330,2025-10-10,2025-10-10 12:08:00,None,None,1.0,0.0,0.0,0.0,...,No Fault,23,24,-1,NaN,NaN,None,Normal,NaN,2
2,16,2025-10-10 12:08:02.430,2025-10-10,2025-10-10 12:08:00,None,None,1.0,0.0,0.0,0.0,...,No Fault,23,24,-1,NaN,NaN,None,Normal,NaN,3
3,16,2025-10-10 12:08:04.810,2025-10-10,2025-10-10 12:08:00,None,None,1.0,0.0,0.0,0.0,...,No Fault,23,24,-1,NaN,NaN,None,Normal,NaN,4
4,16,2025-10-10 12:08:05.130,2025-10-10,2025-10-10 12:08:00,None,None,1.0,0.0,0.0,0.0,...,No Fault,23,24,-1,NaN,NaN,None,Normal,NaN,5


Index(['id', 'timestamp', 'dt', 'ts_mins', 'batterycoolingstate',
       'batterycoolanttemperature', 'ignitionstatus',
       'temperaturedifferencealarm', 'chargingcurrentalarm',
       'dischargecurrentalarm', 'vehiclereadycondition',
       'gun_connection_status', 'ignitionstatus', 'pack1_cellmax_temperature',
       'pack1_maxtemperature_cell_number', 'pack1_cell_min_temperature',
       'pack1_celltemperature_cellnumber', 'cpo_rn', 'id', 'timestamp', 'date',
       'ts_mins_cac', 'b2t_tms_control_cmd', 'b2t_set_water_out_temp',
       'b2t_battery_min_temp', 'b2t_battery_max_temp', 'tms_working_mode',
       'tms_fault_code', 'coolant_out_temp', 'coolant_in_temp',
       'coolant_temp_diff', 'comp_target_freq', 'comp_running_freq',
       'comp_status', 'v2t_vehicle_coolant_low', 'comp_current', 'cac_rn'],
      dtype='object')

170


In [7]:
df.to_csv('bcs_temp.csv')